# Neural Network Evaluation 


In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np

from gerumo.data.dataset import describe_dataset
from gerumo.data.generators import build_generator
from gerumo.utils.engine import (
    setup_cfg, setup_environment, setup_experiment, setup_model, load_model, build_dataset, get_dataset_name
)
from gerumo.utils.structures import Event, Task
from gerumo.models.base import build_model
from gerumo.visualization.metrics import training_history
from gerumo.visualization.samples import event_regression

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

## Select experiment

In [ ]:
# Select a trained model directory
#args['config_file'] = '/mnt/storage-lite/experiments/cta/angular/smooth_experiments/20220604_072240_umonne_lst_cut_l21000'
args['config_file'] = '/mnt/storage-lite/experiments/cta/angular/smooth_experiments/20220604_072240_umonne_lst_full_l2100'
#args['config_file'] = '/mnt/storage-lite/experiments/cta/angular/smooth_experiments/20220604_072240_umonne_lst_full_l210'
#args['config_file'] = '/mnt/storage-lite/experiments/cta/angular/smooth_experiments/20220604_072240_umonne_lst_cut_l210'

#args['config_file'] = '/home/asuka/projects/gerumo2/tools/output/20220329_094830_bmo_mst_regression'

# Select the best batch
args['epoch'] = -1

# Use the validation datasets for evaluation
args['use_validation'] = False

# Select a test datasets (on axis/off axis)
args['opts'] = [
    'OUTPUT_DIR', '/mnt/storage-lite/experiments/cta/angular/smooth_experiments/20220604_072240_umonne_lst_full_l21000',
    'DATASETS.TEST.FOLDER', '/mnt/storage-lite/datasets/cta/Prod5_DL1/gamma-diffuse/test',
    'DATASETS.TEST.EVENTS', '/mnt/storage-lite/datasets/cta/Prod5_DL1/DL1_Prod5_GammaDiffuse_Test/cut_hillas_intensity_1000/events',
    'DATASETS.TEST.TELESCOPES', '/mnt/storage-lite/datasets/cta/Prod5_DL1/DL1_Prod5_GammaDiffuse_Test/cut_hillas_intensity_1000/telescopes'
]

## Setup

In [ ]:
# Find the config.yml
if os.path.isdir(args.config_file):
    args.config_file = os.path.join(args.config_file, 'config.yml')
# Load the configurations
cfg = setup_cfg(args)
output_dir, evaluation_dir = setup_experiment(cfg, training=False)
logger = setup_environment(cfg)

## Load evaluation dataset

In [ ]:
# Setup evaluation datasets directory
if args.use_validation:
    evaluation_dataset_name = 'validation'
else:
    evaluation_dataset_name = 'test'
evaluation_dir = evaluation_dir / evaluation_dataset_name
evaluation_dir.mkdir(exist_ok=True)

# Build evaluation dataset
evaluation_dataset = build_dataset(cfg, evaluation_dataset_name)
describe_dataset(evaluation_dataset, logger, save_to=evaluation_dir / 'description.txt')

In [ ]:
evaluation_dataset

## Build generator

In [ ]:
evaluation_generator = build_generator(cfg, evaluation_dataset)

## Load model

In [ ]:
#%%capture
input_shape = evaluation_generator.get_input_shape()
model = build_model(cfg, input_shape)
model.summary()
model = load_model(model, evaluation_generator, output_dir, args.epoch)

## Start evaluation

In [ ]:
n_samples = 10
random_state = np.random.RandomState(18061996)
batch_samples = random_state.randint(len(evaluation_generator) - 1, size=n_samples)
batch_samples = [3, 5]

In [ ]:
events = []
uncertainties = []
for i, (X, event_true) in enumerate(tqdm(evaluation_generator)):
    predictions, y, uncertainty = model(X, uncertainty=True)
    event_predictions = Event.add_prediction_list(event_true, predictions, model.task)
    events += event_predictions
    uncertainties += [u for u in uncertainty.numpy()]
    if (model.task is Task.REGRESSION) and (i in batch_samples):
        j = random_state.randint(len(X))
        targets = cfg.OUTPUT.REGRESSION.TARGETS
        targets_domains = cfg.OUTPUT.REGRESSION.TARGETS_DOMAINS
        # Plot input
        input_observation = X[j]
        # Plot event prediction
        event_prediction = event_predictions[j]
        model_output = y[j]
        event_regression(event_prediction, model_output, model.REGRESSION_OUTPUT_TYPE, targets, targets_domains)
evaluation_results = Event.list_to_dataframe(events)
evaluation_results['uncertainty'] = uncertainties
evaluation_results.to_csv(evaluation_dir / 'results.csv', index=False)

In [ ]:
evaluation_results

## Plot results

In [ ]:
from gerumo.visualization import metrics
from pathlib import Path

In [ ]:
evaluation_results = pd.read_csv(evaluation_dir / 'results.csv')
evaluation_results

In [ ]:
if model.task is Task.REGRESSION:
    # Target Regression
    targets = [t.split('_')[1] for t in cfg.OUTPUT.REGRESSION.TARGETS]
    metrics.targets_regression(evaluation_results, targets)
    # Resolution
    metrics.reconstruction_resolution(evaluation_results, targets, ylim=(0, 2))
    # Theta2 distribution
    metrics.theta2_distribution(evaluation_results, targets)

In [ ]:
if model.task is Task.CLASSIFICATION:
    # Classification Report
    labels = evaluation_generator.output_mapper.classes
    metrics.classification_report(evaluation_results.pred_class_id, evaluation_results.true_class_id, labels=labels)
    metrics.confusion_matrix(evaluation_results.pred_class_id, evaluation_results.true_class_id, labels=labels)


# Sample Visualization

In [ ]:
n_samples = 10

In [ ]:
# Select random batch
batch_i = np.random.randint(len(evaluation_generator))
X, event_true = evaluation_generator[batch_i]

# Prediction
predictions, y, uncertainties = model(X, uncertainty=True)
event_predictions = Event.add_prediction_list(event_true, predictions, model.task)

In [ ]:
samples_j = np.random.randint(len(X), size=n_samples)
if model.task is Task.REGRESSION:
    targets = cfg.OUTPUT.REGRESSION.TARGETS
    targets_domains = cfg.OUTPUT.REGRESSION.TARGETS_DOMAINS
    for j in samples_j:
        # Plot input
        input_observation = X[j]
        # Plot event prediction
        event_prediction = event_predictions[j]
        model_output = y[j]
        event_regression(event_prediction, model_output, model.REGRESSION_OUTPUT_TYPE, targets, targets_domains)
        # Plot uncertainty
        uncertainty = uncertainties[j]

In [ ]:
if model.task is Task.CLASSIFICATION:
    pass